In [271]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy


Using TensorFlow backend.


In [272]:
def extract(sudoku_loc):
    
    img_path = sudoku_loc
    img = cv2.imread(img_path)
    he, wi, c = img.shape
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # (2) threshold-inv and morph-open 
    th, threshed = cv2.threshold(img, 220, 255, cv2.THRESH_BINARY_INV)
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_OPEN, np.ones((2,2)))

    # (3) find and filter contours, then draw on src 
    cnts = cv2.findContours(morphed, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]


    digitCnts=[]
    counter=0
    for cnt in cnts:
        x,y,w,h  = cv2.boundingRect(cnt)
        if ((w >= wi/11 and w<=wi/7.8) and (h >= he/11 and h <= he/7.8)):
            digitCnts.append([x,y,w,h])
            counter+=1
            cv2.rectangle(morphed, (x,y), (x+w, y+h), (0, 0, 255), 2, cv2.LINE_AA)
    print(counter)  

    cv2.imwrite("morphed.png", morphed)

    img=morphed
    for i in range (81):
        pts = digitCnts[i]
        x1=pts[0]
        y1=pts[1]
        w1=pts[2]
        h1=pts[3]
        crop_img = img[x1:x1+h1, y1:y1+w1]
        cv2.imwrite("D:\\duke\\project\\sudoku\\cropped\\"+str(i)+".png", crop_img)

In [273]:
def predict(loc,mod):

    #load the saved model
    model = load_model(mod)

    model.compile(optimizer=Adam(learning_rate=0.001), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

    matrix=[]
    for i in range(81):
        #loading images from your system
        path=loc + str(i) +'.png' 
        img1 = cv2.imread(path)

        #converting the image to grayscale
        img= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

        #resizing the image
        img = cv2.resize(img,(28,28))
        img = np.reshape(img,[1,28,28,1])
        img1 = np.reshape(img,[28,28,1])

        img1= img1[10:20, 10:20] 

        count=0
        for i in range(10):
            for j in range(10):
                if (img1[i][j][0]==0):
                        count+=1
        if(count>80):
            matrix.append(0)
        else:

            #predicting the model
            pre = model.predict(img)
            matrix.append(np.argmax(pre))
        

    arr = to_matrix(matrix,9)
    a=matrixflip(arr)
    return(a)

def to_matrix(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

def matrixflip(tempm):
    
    for i in range(0,len(tempm),1):
            tempm[i].reverse()
    
    tempm.reverse()
    tempm=np.transpose(tempm)
    return(tempm)





In [274]:
img_path = 'D:\\duke\\project\\sudoku\\sudoku3.jpg'
img = cv2.imread(img_path)
extract(img_path)
cropped_path ='D:\\duke\\project\\sudoku\\cropped\\'
weights_path='D:\\duke\\project\\sudoku\\model1.h5'
board = predict(cropped_path,weights_path)

board=board.tolist()



81
(81, 4)


In [275]:
def solve(sudoku):
    find = find_empty(sudoku)
    if not find:
        return True
    else:
        row,col = find
    for i in range(1,10):
        if valid(sudoku,i,(row,col)):
            sudoku[row][col]=i
            if solve(sudoku):
                return True
            sudoku[row][col]=0
    return False


def print_board(sudoku):
    for i in range(len(sudoku[0])):
        if(i%3 == 0 and i!=0):
            print("- - - - - - - - - - - - - - - - ")
        for j in range(len(sudoku[1])):
            if(j%3 == 0 and j!=0):
                print("| ",end =" ")
            if (j==8):
                print(sudoku[i][j])
            else:
                print(str(sudoku[i][j]) + " ",end=" ")
def find_empty(sudoku):
    for i in range(len(sudoku[0])):
        for j in range(len(sudoku[1])):
            if(sudoku[i][j]==0):
                return (i,j)
    return None

def valid(sudoku,num,pos):
    #check row
    for i in range(len(sudoku[0])):
        if(sudoku[pos[0]][i]==num and pos[1]!=i):
            return False

    #check column 
    for i in range(len(sudoku[1])):
        if(sudoku[i][pos[1]]==num and pos[0]!=i):
            return False
    #check box
    box_x = pos[0]//3
    box_y = pos[1]//3
    for i in range(box_x*3,box_x*3 +3):
        for j in range(box_y*3,box_y*3 +3):
            if(sudoku[i][j]== num and (i,j) != pos):
                return False
    return True




In [ ]:
plt.imshow(img)
plt.axis(False)

In [276]:
print_board(board)
solve(board)
print("--------------------------------------------")
print("--------------------------------------------")
print_board(board)

8  0  0  |  7  1  5  |  0  0  4
0  0  5  |  3  0  6  |  7  0  0
3  0  6  |  4  0  8  |  9  0  1
- - - - - - - - - - - - - - - - 
0  6  0  |  0  5  0  |  0  4  0
0  0  0  |  8  0  7  |  0  0  0
0  5  0  |  0  4  0  |  0  9  0
- - - - - - - - - - - - - - - - 
6  0  9  |  5  0  3  |  4  0  2
0  0  4  |  9  0  2  |  5  0  0
5  0  0  |  1  6  4  |  0  0  9
--------------------------------------------
--------------------------------------------
8  9  2  |  7  1  5  |  6  3  4
4  1  5  |  3  9  6  |  7  2  8
3  7  6  |  4  2  8  |  9  5  1
- - - - - - - - - - - - - - - - 
7  6  8  |  2  5  9  |  1  4  3
9  4  1  |  8  3  7  |  2  6  5
2  5  3  |  6  4  1  |  8  9  7
- - - - - - - - - - - - - - - - 
6  8  9  |  5  7  3  |  4  1  2
1  3  4  |  9  8  2  |  5  7  6
5  2  7  |  1  6  4  |  3  8  9
